In [ ]:
# external imports
import os
import re
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from matplotlib import ticker as mticker
from pymatgen.core.structure import Structure

# setup for plots
plt.rcParams["text.usetex"] = True
plt.rc("font", family="serif")
plt.rc("xtick", labelsize="x-small")
plt.rc("ytick", labelsize="x-small")
plt.rc("text", usetex=True)

# so python can find the calc directory
os.chdir("../")

The state-of-the-art (SOTA) algorithm mentioned in the paper is labeled
"npj" instead of "sota" in the code for historical reasons.

In [ ]:
# data structures that contain all relevant information
@dataclass
class txt_data:
    id: str  # materials project id
    name: str  # material name
    structure: Structure  # pymatgen structure class
    data: list  # output of the .txt file
    gwc: list  # list with all output classes


@dataclass
class cdata:
    id: str  # materials project id
    name: str  # material name
    structure: Structure  # pymatgen structure class
    kppa: np.array  # k-point density
    kpt_str: list  # k-point string
    nkir: np.array  # number of irreducible k-points
    params: np.ndarray  # gw parameters
    gap: np.array  # direct gap at the gamma point
    ngw: np.ndarray  # total number of gw calculations
    tcpu: np.array  # cpu time
    gwc: list  # list with all output classes
    extra: np.array  # gap extrapolation from the fit algorithm

In [ ]:
# parser for my txt files
def output_parser(txt_path, calc_type):
    with open(txt_path, "r") as f:
        s = f.readlines()
    s = s[1:]  # skip the header
    if calc_type == "cs":
        data = np.array(-1 * np.ones([5, 10], dtype=int), dtype=str)
    elif calc_type == "npj":
        data = np.array(-1 * np.ones([5, 11], dtype=int), dtype=str)
    elif calc_type == "ref":
        data = np.array(-1 * np.ones([5, 9], dtype=int), dtype=str)
    for i, l in enumerate(s):
        if not "FAILED" in l:
            data[i] = l.split()
    return data

In [ ]:
"""
Parse the results for all 60 bulk materials
"""

# initializations
calc_dir = "calc"
mp_ids = os.listdir(calc_dir)
materials_2d = os.listdir("input_2d")
materials_2d = [x.split(".")[0] for x in materials_2d]
[mp_ids.remove(f) for f in materials_2d]
res_cs = []
res_npj = []
res_ref = []
conv_thr = 25  # meV

# to satisfy my own OCD we sort by the mp-ids
sort_idx = np.argsort([int(x.split("-")[1]) for x in mp_ids])
mp_ids = np.array(mp_ids)[sort_idx]

# array for the table in the supplement
ids = []  # sort sorting...
si_table = []

# loop over all materials
for mp in mp_ids:
    # materials project id
    id = re.findall(r"mp-\d+", mp)[0]
    ids.append(int(id.split("-")[1]))

    # parse the structure
    with open(f"{calc_dir}/{mp}/structure.pckl", "rb") as f:
        structure, name = pickle.load(f)
    for site in structure.sites:  # fix the site labels ...
        site.label = site.species.elements[0].name

    # collect the data from the cooridnate search algorithm
    cs_path = f"{calc_dir}/{mp}/yambo_g0w0_cs_kpt"
    if os.path.isfile(f"{cs_path}/{mp}_gw_conv_cs.txt"):
        # data from the output text file
        data = output_parser(f"{cs_path}/{mp}_gw_conv_cs.txt", "cs")

        # collect all classes
        cs_class = []
        for kppa in data[:, 0]:
            cs_class_path = f"{cs_path}/kppa{kppa}_cs/g0w0_cs/class_cs.pckl"
            if os.path.isfile(cs_class_path):
                with open(cs_class_path, "rb") as f:
                    gw_class = pickle.load(f)
            else:
                gw_class = 0
            cs_class.append(gw_class)

        # adjust the name of the material for the figure titles to get the correct latex formatting
        name = name.replace("2", "$_2$")  # we only have elements with 2 in the name

        # create the dataclass for each k-grid
        res_cs.append(
            cdata(
                id,  # materials project id
                name,  # material name
                structure,  # pymatgen structure class
                data[:, 0].astype(int),  # kppa
                data[:, 1],  # k-point string
                data[:, 2].astype(int),  # number of irreducible k-points
                data[:, 3:6].astype(int),  # gw parameteres
                data[:, 6].astype(float),  # direct gap
                data[:, 7].astype(int),  # number of gw calculations
                data[:, 8].astype(int) * data[:, 9].astype(int),  # cpu time
                cs_class,  # list with all classes
                np.zeros(5),  # gap extrapolation from the fit algorithm
            )
        )

    else:
        print(f"{mp} cs is missing...")

    # temporary list for the si table
    # mp-id, name, cs Nb, cs Gcut, cs gap
    temp_si = [
        id,
        name,
        data[-1, 4].astype(int),
        data[-1, 5].astype(int),
        f"{data[-1,6].astype(float):.3f}",
    ]

    # collect the data from the npj fit algorithm
    npj_path = f"{calc_dir}/{mp}/yambo_g0w0_npj_kpt/"
    if os.path.isfile(f"{npj_path}/{mp}_gw_conv_npj.txt"):
        # data from the output text file
        data = output_parser(f"{npj_path}/{mp}_gw_conv_npj.txt", "npj")

        # collect all classes
        npj_class = []
        extra = []
        for kppa in data[:, 0]:
            npj_class_path = f"{npj_path}/kppa{kppa}_npj/g0w0_npj/class_npj.pckl"
            if os.path.isfile(npj_class_path):
                with open(npj_class_path, "rb") as f:
                    gw_class = pickle.load(f)
            else:
                gw_class = 0
            npj_class.append(gw_class)
            if gw_class == 0:
                extra.append(-1)
            else:
                extra.append(gw_class.extra)
        extra = np.array(extra)

        # create the dataclass for each k-grid
        res_npj.append(
            cdata(
                id,  # materials project id
                name,  # material name
                structure,  # pymatgen structure class
                data[:, 0].astype(int),  # kppa
                data[:, 1],  # k-point string
                data[:, 2].astype(int),  # number of irreducible k-points
                data[:, 3:6].astype(int),  # gw parameteres
                data[:, 6].astype(float),  # direct gap
                data[:, 7].astype(int),  # number of gw calculations
                data[:, 8].astype(int) * data[:, 9].astype(int),  # cpu time
                npj_class,  # list with all classes
                extra,  # gap extrapolation from the fit algorithm
            )
        )
    else:
        print(f"{mp} npj is missing...")

    # append the npj gap (if the algorithm converged...)
    if data[-1, 6].astype(float) == -1:
        temp_si.append(f"-")
    else:
        temp_si.append(f"{data[-1,6].astype(float):.3f}")

    # collect the data from the cs reference calculations with high computation parameters
    ref_path = f"{calc_dir}/{mp}/yambo_g0w0_cs_reference/"
    if os.path.isfile(f"{ref_path}/{mp}_gw_conv_cs_ref.txt"):
        # data from the output text file
        data = output_parser(f"{ref_path}/{mp}_gw_conv_cs_ref.txt", "ref")

        # create the dataclass for each k-grid
        res_ref.append(
            cdata(
                id,  # materials project id
                name,  # material name
                structure,  # pymatgen structure class
                data[:, 0].astype(int),  # kppa
                data[:, 1],  # k-point string
                data[:, 2].astype(int),  # number of irreducible k-points
                data[:, 3:6].astype(int),  # gw parameteres
                data[:, 6].astype(float),  # direct gap
                np.ones(5),  # number of gw calculations
                data[:, 7].astype(int) * data[:, 8].astype(int),  # cpu time
                [],  # list with all classes
                np.zeros(5),  # gap extrapolation from the fit algorithm
            )
        )
    else:
        print(f"{mp} ref is missing...")

    # append the ref gap
    temp_si.append(f"{data[-1,6].astype(float):.3f}")

    # append the si table
    si_table.append(temp_si)

# turn the si table into a numpy array
si_table = np.array(si_table)

# number of finished calculations
print(f"#cs  = {len(res_cs)}")
print(f"#npj = {len(res_npj)}")
print(f"#ref = {len(res_ref)}")

In [ ]:
# swap name and id column
si_table[:, [0, 1]] = si_table[:, [1, 0]]

# create a dataframe and convert it to a latex table
var_names = [
    "ID",
    "$N_b$ [CS]",
    "$G_\\textrm{cut}$~(Ry) [CS]",
    "$\\textrm{E}_\\textrm{gap}^{{\\Gamma}-{\\Gamma}}$~(eV) [CS]",
    "$\\textrm{E}_\\textrm{gap}^{{\\Gamma}-{\\Gamma}}$~(eV) [SOTA]",
    "$\\textrm{E}_\\textrm{gap}^{{\\Gamma}-{\\Gamma}}$~(eV) [R]",
]
latex_table = pd.DataFrame(si_table[:, 1:], columns=var_names, index=si_table[:, 0])
latex_table_style = latex_table.style
print(
    latex_table_style.to_latex(
        column_format="cccccc",
        position="h",
        position_float="centering",
        hrules=True,
        label="mp",
        caption="",
    )
)

In [ ]:
# check whether the gaps in the table are converged with respect to the k-point grid
# for this we check the k-point convergence for the calculations with the large parameters in W (ref)
for r in res_ref:
    if np.abs(r.gap[-1] - r.gap[-2]) * 1000 > conv_thr:
        print(
            f"{r.id} {r.name}"
        )  # these materials are highlighted in the table in the SI

In [ ]:
"""
Function to combine two reference convergence calculations with different parameter in one figure
"""


def ref_calc_subfigure(mat_names, fig_file_name):
    # material figure path from the reference convergence calculations
    mat_fig_path = [
        f"ref/{mat_name}/g0w0_npj/gw_conv_plt_npj_algo.png" for mat_name in mat_names
    ]

    # figure size (tested)
    width = 2 * 3.35
    height = 6

    # figure
    fig, axs = plt.subplots(1, 2, figsize=(width, height), facecolor="w", dpi=600)

    # add (a), (b) to the subplots
    axs[0].text(-0.0, 1.0, r"\textbf{a}", transform=axs[0].transAxes, size=11)
    axs[1].text(-0.0, 1.0, r"\textbf{b}", transform=axs[1].transAxes, size=11)

    # pngs
    axs[0].imshow(plt.imread(mat_fig_path[0]))
    axs[1].imshow(plt.imread(mat_fig_path[1]))

    for ax in axs:
        for side in ("top", "left", "bottom", "right"):
            ax.spines[side].set_visible(False)
            ax.tick_params(
                left=False,
                right=False,
                labelleft=False,
                labelbottom=False,
                bottom=False,
            )

    # fix layout
    fig.tight_layout()

    # save figure
    fig.savefig(f"plots/supplement/{fig_file_name}.pdf", bbox_inches="tight")

In [ ]:
# diamond with SG15 and 24 Ry FFTGvecs
ref_calc_subfigure(
    ["mp-66_fftgvecs_24", "mp-66_SG15_1_fftgvecs_24"], "diamond_ref_SG15_fftgvecs_24"
)

# diamond with SG15 and all FFTGvecs
ref_calc_subfigure(["mp-66", "mp-66_SG15_1"], "diamond_ref_SG15")

# silicon with SG15 and all FFTGvecs
ref_calc_subfigure(["mp-149", "mp-149_SG15_1"], "silicon_ref_SG15")

# hBN with SG15 and all FFTGvecs
ref_calc_subfigure(["mp-984", "mp-984_SG15_1"], "hbn_ref_SG15")

Influence of the CS starting point

In [ ]:
"""
Parse the results for the 10 bulk materials which 
were started with a different CS starting point
"""

# initializations
calc_dir = "calc"
mp_ids = os.listdir(calc_dir)
materials_2d = os.listdir("input_2d")
materials_2d = [x.split(".")[0] for x in materials_2d]
[mp_ids.remove(f) for f in materials_2d]
res_cs_higher_start = []
conv_thr = 25  # meV

# to satisfy my own OCD we sort by the mp-ids
sort_idx = np.argsort([int(x.split("-")[1]) for x in mp_ids])
mp_ids = np.array(mp_ids)[sort_idx]

# indicies to find the cs calculations with the same mp-id from the other list
idx_cs = []

# loop over all materials
for i, mp in enumerate(mp_ids):
    # materials project id
    id = re.findall(r"mp-\d+", mp)[0]

    # parse the structure
    with open(f"{calc_dir}/{mp}/structure.pckl", "rb") as f:
        structure, name = pickle.load(f)
    for site in structure.sites:  # fix the site labels ...
        site.label = site.species.elements[0].name

    # collect the data from the cooridnate search algorithm
    cs_path = f"{calc_dir}/{mp}/yambo_g0w0_cs_kpt_higher_start"
    if os.path.isfile(f"{cs_path}/{mp}_gw_conv_cs.txt"):
        # remember the index
        idx_cs.append(i)

        # data from the output text file
        data = output_parser(f"{cs_path}/{mp}_gw_conv_cs.txt", "cs")

        # collect all classes
        cs_class = []
        for kppa in data[:, 0]:
            cs_class_path = f"{cs_path}/kppa{kppa}_cs/g0w0_cs/class_cs.pckl"
            if os.path.isfile(cs_class_path):
                with open(cs_class_path, "rb") as f:
                    gw_class = pickle.load(f)
            else:
                gw_class = 0
            cs_class.append(gw_class)

        # adjust the name of the material for the figure titles to get the correct latex formatting
        name = name.replace("2", "$_2$")  # we only have elements with 2 in the name

        # create the dataclass for each k-grid
        res_cs_higher_start.append(
            cdata(
                id,  # materials project id
                name,  # material name
                structure,  # pymatgen structure class
                data[:, 0].astype(int),  # kppa
                data[:, 1],  # k-point string
                data[:, 2].astype(int),  # number of irreducible k-points
                data[:, 3:6].astype(int),  # gw parameteres
                data[:, 6].astype(float),  # direct gap
                data[:, 7].astype(int),  # number of gw calculations
                data[:, 8].astype(int) * data[:, 9].astype(int),  # cpu time
                cs_class,  # list with all classes
                np.zeros(5),  # gap extrapolation from the fit algorithm
            )
        )

# number of finished calculations
print(f"#cs with higher starting point = {len(res_cs_higher_start)}")

In [ ]:
# check how much the gap differs when we
# start with a different CS starting point
gap_diff_starting_point = []
for i, r in enumerate(res_cs_higher_start):
    gap_diff = 1000 * np.abs(r.gap - res_cs[idx_cs[i]].gap)  # meV
    gap_diff_formatted = [f"{g:.2f}" for g in gap_diff]
    temp_list = np.append(
        [r.name, r.id],
        gap_diff_formatted,
    )
    temp_list = np.append(temp_list, f"{np.mean(gap_diff):.2f}")
    gap_diff_starting_point.append(temp_list)
gap_diff_starting_point = np.array(gap_diff_starting_point)

# print the total average
print(f"Average gap difference with different starting point:")
print(f"{np.mean([float(g) for g in gap_diff_starting_point[:,-1]]):.2f} meV\n")

# create a dataframe and convert it to a latex table
var_names = [
    "ID",
    r"$\mathbf{k_1}$ (meV)",
    r"$\mathbf{k_2}$ (meV)",
    r"$\mathbf{k_3}$ (meV)",
    r"$\mathbf{k_4}$ (meV)",
    r"$\mathbf{k_5}$ (meV)",
    r"$\bar{\mathbf{k}}$ (meV)",
]
latex_table = pd.DataFrame(
    gap_diff_starting_point[:, 1:],
    columns=var_names,
    index=gap_diff_starting_point[:, 0],
)
latex_table_style = latex_table.style
print(
    latex_table_style.to_latex(
        column_format="cccccccc",
        position="h",
        position_float="centering",
        hrules=True,
        label="tab:starting_point",
        caption="",
    )
)

Check how the W convergence depends on the k-point grid.
But now we do this for the other k-point grids.

In [ ]:
# step size in our algorithm
delta_cs = [100, 4]
delta_npj = [200, 4]

# calculate the relative parameters change in W from the k2 to final k-point grid
# normalized to the step size of the respective convergence algorithm
w_diff_cs = []
w_diff_npj = []
for r in res_cs:
    if not -1 in r.params[:, 0]:
        temp_w_diff_cs = (r.params[1, 1:] - r.params[-1, 1:]) / delta_cs
        w_diff_cs.append(temp_w_diff_cs)
    else:
        print(f"Skipping: {r.id} ({r.name})")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_cs[0] < 0:
        print(f"CS Negative N_b:    {r.id:<10s} {r.name:<10s} {temp_w_diff_cs[0]}")
    if temp_w_diff_cs[1] < 0:
        print(f"CS Negative G_cut:  {r.id:<10s} {r.name:<10s} {temp_w_diff_cs[1]}")
for r in res_npj:
    if not -1 in r.params[:, 0]:
        temp_w_diff_npj = (r.params[1, 1:] - r.params[-1, 1:]) / delta_npj
        w_diff_npj.append(temp_w_diff_npj)
    else:
        print(f"Skipping: {r.id} ({r.name})")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_npj[0] < 0:
        print(f"NPJ Negative N_b:   {r.id:<10s} {r.name:<10s} {temp_w_diff_npj[0]}")
    if temp_w_diff_npj[1] < 0:
        print(f"NPJ Negative G_cut: {r.id:<10s} {r.name:<10s} {temp_w_diff_npj[1]}")
w_diff_cs = np.array(w_diff_cs)
w_diff_npj = np.array(w_diff_npj)

In [ ]:
# figure size (tested)
width = 2 * 2.65
height = 4.5

# text size
fs = 10

# figure
fig, axs = plt.subplots(2, 2, figsize=(width, height), facecolor="w", dpi=600)

# add (a), (b), (c) and (d) to the subplots
axs[0, 0].text(-0.2, 1.15, r"\textbf{a}", transform=axs[0, 0].transAxes, size=11)
axs[0, 1].text(-0.18, 1.15, r"\textbf{b}", transform=axs[0, 1].transAxes, size=11)
axs[1, 0].text(-0.2, 1.15, r"\textbf{c}", transform=axs[1, 0].transAxes, size=11)
axs[1, 1].text(-0.18, 1.15, r"\textbf{d}", transform=axs[1, 1].transAxes, size=11)

# plot the change in the number of bands when comparing the 2nd k-grid convergence to the last k-grid
axs[0, 0].hist(
    w_diff_cs[:, 0],
    bins=np.arange(np.min(w_diff_cs[:, 0]), np.max(w_diff_cs[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 0].axvline(x=np.mean(w_diff_cs[:, 0]), color="magenta", linestyle="--")
axs[0, 0].axvline(x=np.median(w_diff_cs[:, 0]), color="limegreen", linestyle="--")
axs[0, 1].hist(
    w_diff_npj[:, 0],
    bins=np.arange(np.min(w_diff_npj[:, 0]), np.max(w_diff_npj[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 1].axvline(x=np.mean(w_diff_npj[:, 0]), color="magenta", linestyle="--")
axs[0, 1].axvline(x=np.median(w_diff_npj[:, 0]), color="limegreen", linestyle="--")

# plot the change in the cutoff when comparing the 2nd k-grid convergence to the last k-grid
axs[1, 0].hist(
    w_diff_cs[:, 1],
    bins=np.arange(np.min(w_diff_cs[:, 1]), np.max(w_diff_cs[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 0].axvline(x=np.mean(w_diff_cs[:, 1]), color="magenta", linestyle="--")
axs[1, 0].axvline(x=np.median(w_diff_cs[:, 1]), color="limegreen", linestyle="--")
axs[1, 1].hist(
    w_diff_npj[:, 1],
    bins=np.arange(np.min(w_diff_npj[:, 1]), np.max(w_diff_npj[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 1].axvline(x=np.mean(w_diff_npj[:, 1]), color="magenta", linestyle="--")
axs[1, 1].axvline(x=np.median(w_diff_npj[:, 1]), color="limegreen", linestyle="--")

# titles
axs[0, 0].set_title(r"CS", fontsize=fs + 2, pad=15)
axs[0, 1].set_title(r"SOTA", fontsize=fs + 2, pad=15)

# axis labels
axs[0, 0].set_xlabel(r"$C^{k_2}_{N_b}$", size=fs)
axs[0, 1].set_xlabel(r"$C^{k_2}_{N_b}$", size=fs)
axs[0, 0].set_ylabel(r"Occurrences", size=fs)
axs[1, 0].set_xlabel(r"$C^{k_2}_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 1].set_xlabel(r"$C^{k_2}_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 0].set_ylabel(r"Occurrences", size=fs)

# axis ticks
axs[0, 0].xaxis.set_tick_params(labelsize=fs)
axs[0, 0].yaxis.set_tick_params(labelsize=fs)
axs[0, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 0].yaxis.set_ticks(axs[0, 0].get_yticks().astype(int))
axs[0, 0].xaxis.set_ticks([-3, -2, -1, 0, 1, 2, 3, 4, 5])
axs[0, 0].set_xlim([-3, 5])
axs[0, 1].xaxis.set_tick_params(labelsize=fs)
axs[0, 1].yaxis.set_tick_params(labelsize=fs)
axs[0, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 1].yaxis.set_ticks(axs[0, 1].get_yticks().astype(int))
axs[0, 1].set_xlim([-3, 5])
axs[0, 1].xaxis.set_ticks([-3, -2, -1, 0, 1, 2, 3, 4, 5])
axs[1, 0].xaxis.set_tick_params(labelsize=fs)
axs[1, 0].yaxis.set_tick_params(labelsize=fs)
axs[1, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 0].set_xlim([-7, 5])
axs[1, 0].xaxis.set_ticks([-6, -4, -2, 0, 2, 4])
axs[1, 0].yaxis.set_ticks([0, 10, 20, 30, 40])
axs[1, 1].xaxis.set_tick_params(labelsize=fs)
axs[1, 1].yaxis.set_tick_params(labelsize=fs)
axs[1, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 1].set_xlim([-7, 5])
axs[1, 1].xaxis.set_ticks([-6, -4, -2, 0, 2, 4])
axs[1, 1].yaxis.set_ticks([0, 10, 20, 30, 40])

# fix layout
plt.tight_layout()
plt.subplots_adjust(hspace=0.6, wspace=0.25)

# save the figure
fig.savefig("plots/supplement/param_indep_w_k_2.pdf")

In [ ]:
# check the percentage of how many times the k2 grid is > 0,
# i.e. that the convergence surface is more flat on the final k-point grid compared to the gamma-only grid
print("CS: C_p >= 0")
print(
    f"{len(w_diff_cs[(w_diff_cs[:,0] >= 0) & (w_diff_cs[:,1] >= 0), 0]) / len(w_diff_cs[:,0]) * 100:.2f} %\n"
)
print("NPJ: C_p >= 0")
print(
    f"{len(w_diff_npj[(w_diff_npj[:,0] >= 0) & (w_diff_npj[:,1] >= 0), 0]) / len(w_diff_npj[:,0]) * 100:.2f} %\n"
)
# we need to check where C_P >= 0 holds for both parameters at the same time

# some statistics for the parameter stability of W with respect to the k-point grid density
print("CS: MEAN")
print(f"N_b:   {np.mean(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_cs[:,1]):.1f}\n")

print("CS: MEDIAN")
print(f"N_b:   {np.median(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_cs[:,1]):.1f}\n")

print("CS: RANGE")
print(f"N_b:   {np.max(w_diff_cs[:,0]) - np.min(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_cs[:,1]) - np.min(w_diff_cs[:,1]):.1f}\n")

print("NPJ: MEAN")
print(f"N_b:   {np.mean(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_npj[:,1]):.1f}\n")

print("NPJ: MEDIAN")
print(f"N_b:   {np.median(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_npj[:,1]):.1f}\n")

print("NPJ: RANGE")
print(f"N_b:   {np.max(w_diff_npj[:,0]) - np.min(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_npj[:,1]) - np.min(w_diff_npj[:,1]):.1f}")

In [ ]:
# step size in our algorithm
delta_cs = [100, 4]
delta_npj = [200, 4]

# calculate the relative parameters change in W from the k3 to final k-point grid
# normalized to the step size of the respective convergence algorithm
w_diff_cs = []
w_diff_npj = []
for r in res_cs:
    if not -1 in r.params[:, 0]:
        temp_w_diff_cs = (r.params[2, 1:] - r.params[-1, 1:]) / delta_cs
        w_diff_cs.append(temp_w_diff_cs)
    else:
        print(f"Skipping: {r.id} ({r.name})")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_cs[0] < 0:
        print(f"CS Negative N_b:    {r.id:<10s} {r.name:<10s} {temp_w_diff_cs[0]}")
    if temp_w_diff_cs[1] < 0:
        print(f"CS Negative G_cut:  {r.id:<10s} {r.name:<10s} {temp_w_diff_cs[1]}")
for r in res_npj:
    if not -1 in r.params[:, 0]:
        temp_w_diff_npj = (r.params[2, 1:] - r.params[-1, 1:]) / delta_npj
        w_diff_npj.append(temp_w_diff_npj)
    else:
        print(f"Skipping: {r.id} ({r.name})")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_npj[0] < 0:
        print(f"NPJ Negative N_b:   {r.id:<10s} {r.name:<10s} {temp_w_diff_npj[0]}")
    if temp_w_diff_npj[1] < 0:
        print(f"NPJ Negative G_cut: {r.id:<10s} {r.name:<10s} {temp_w_diff_npj[1]}")
w_diff_cs = np.array(w_diff_cs)
w_diff_npj = np.array(w_diff_npj)

In [ ]:
# figure size (tested)
width = 2 * 2.65
height = 4.5

# text size
fs = 10

# figure
fig, axs = plt.subplots(2, 2, figsize=(width, height), facecolor="w", dpi=600)

# add (a), (b), (c) and (d) to the subplots
axs[0, 0].text(-0.2, 1.15, r"\textbf{a}", transform=axs[0, 0].transAxes, size=11)
axs[0, 1].text(-0.18, 1.15, r"\textbf{b}", transform=axs[0, 1].transAxes, size=11)
axs[1, 0].text(-0.2, 1.15, r"\textbf{c}", transform=axs[1, 0].transAxes, size=11)
axs[1, 1].text(-0.18, 1.15, r"\textbf{d}", transform=axs[1, 1].transAxes, size=11)

# plot the change in the number of bands when comparing the 3rd k-grid convergence to the last k-grid
axs[0, 0].hist(
    w_diff_cs[:, 0],
    bins=np.arange(np.min(w_diff_cs[:, 0]), np.max(w_diff_cs[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 0].axvline(x=np.mean(w_diff_cs[:, 0]), color="magenta", linestyle="--")
axs[0, 0].axvline(x=np.median(w_diff_cs[:, 0]), color="limegreen", linestyle="--")
axs[0, 1].hist(
    w_diff_npj[:, 0],
    bins=np.arange(np.min(w_diff_npj[:, 0]), np.max(w_diff_npj[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 1].axvline(x=np.mean(w_diff_npj[:, 0]), color="magenta", linestyle="--")
axs[0, 1].axvline(x=np.median(w_diff_npj[:, 0]), color="limegreen", linestyle="--")

# plot the change in the cutoff when comparing the 3rd k-grid convergence to the last k-grid
axs[1, 0].hist(
    w_diff_cs[:, 1],
    bins=np.arange(np.min(w_diff_cs[:, 1]), np.max(w_diff_cs[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 0].axvline(x=np.mean(w_diff_cs[:, 1]), color="magenta", linestyle="--")
axs[1, 0].axvline(x=np.median(w_diff_cs[:, 1]), color="limegreen", linestyle="--")
axs[1, 1].hist(
    w_diff_npj[:, 1],
    bins=np.arange(np.min(w_diff_npj[:, 1]), np.max(w_diff_npj[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 1].axvline(x=np.mean(w_diff_npj[:, 1]), color="magenta", linestyle="--")
axs[1, 1].axvline(x=np.median(w_diff_npj[:, 1]), color="limegreen", linestyle="--")

# titles
axs[0, 0].set_title(r"CS", fontsize=fs + 2, pad=15)
axs[0, 1].set_title(r"SOTA", fontsize=fs + 2, pad=15)

# axis labels
axs[0, 0].set_xlabel(r"$C^{k_3}_{N_b}$", size=fs)
axs[0, 1].set_xlabel(r"$C^{k_3}_{N_b}$", size=fs)
axs[0, 0].set_ylabel(r"Occurrences", size=fs)
axs[1, 0].set_xlabel(r"$C^{k_3}_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 1].set_xlabel(r"$C^{k_3}_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 0].set_ylabel(r"Occurrences", size=fs)

# axis ticks
axs[0, 0].xaxis.set_tick_params(labelsize=fs)
axs[0, 0].yaxis.set_tick_params(labelsize=fs)
axs[0, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 0].yaxis.set_ticks(axs[0, 0].get_yticks().astype(int))
axs[0, 0].xaxis.set_ticks([-2, -1, 0, 1, 2, 3, 4])
axs[0, 0].set_xlim([-2, 4])
axs[0, 1].xaxis.set_tick_params(labelsize=fs)
axs[0, 1].yaxis.set_tick_params(labelsize=fs)
axs[0, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 1].yaxis.set_ticks(axs[0, 1].get_yticks().astype(int))
axs[0, 1].xaxis.set_ticks([-2, -1, 0, 1, 2, 3, 4])
axs[0, 1].set_xlim([-2, 4])
axs[1, 0].xaxis.set_tick_params(labelsize=fs)
axs[1, 0].yaxis.set_tick_params(labelsize=fs)
axs[1, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 0].set_xlim([-4, 4])
axs[1, 0].xaxis.set_ticks([-4, -2, 0, 2, 4])
axs[1, 0].yaxis.set_ticks([0, 10, 20, 30, 40, 50, 60])
axs[1, 1].xaxis.set_tick_params(labelsize=fs)
axs[1, 1].yaxis.set_tick_params(labelsize=fs)
axs[1, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 1].set_xlim([-4, 4])
axs[1, 1].xaxis.set_ticks([-4, -2, 0, 2, 4])
axs[1, 1].yaxis.set_ticks([0, 10, 20, 30, 40, 50])

# fix layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.6, wspace=0.25)

# save the figure
fig.savefig("plots/supplement/param_indep_w_k_3.pdf")

In [ ]:
# check the percentage of how many times the k3 grid is > 0,
# i.e. that the convergence surface is more flat on the final k-point grid compared to the gamma-only grid
print("CS: C_p >= 0")
print(
    f"{len(w_diff_cs[(w_diff_cs[:,0] >= 0) & (w_diff_cs[:,1] >= 0), 0]) / len(w_diff_cs[:,0]) * 100:.2f} %\n"
)
print("NPJ: C_p >= 0")
print(
    f"{len(w_diff_npj[(w_diff_npj[:,0] >= 0) & (w_diff_npj[:,1] >= 0), 0]) / len(w_diff_npj[:,0]) * 100:.2f} %\n"
)
# we need to check where C_P >= 0 holds for both parameters at the same time

# some statistics for the parameter stability of W with respect to the k-point grid density
print("CS: MEAN")
print(f"N_b:   {np.mean(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_cs[:,1]):.1f}\n")

print("CS: MEDIAN")
print(f"N_b:   {np.median(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_cs[:,1]):.1f}\n")

print("CS: RANGE")
print(f"N_b:   {np.max(w_diff_cs[:,0]) - np.min(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_cs[:,1]) - np.min(w_diff_cs[:,1]):.1f}\n")

print("NPJ: MEAN")
print(f"N_b:   {np.mean(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_npj[:,1]):.1f}\n")

print("NPJ: MEDIAN")
print(f"N_b:   {np.median(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_npj[:,1]):.1f}\n")

print("NPJ: RANGE")
print(f"N_b:   {np.max(w_diff_npj[:,0]) - np.min(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_npj[:,1]) - np.min(w_diff_npj[:,1]):.1f}")

In [ ]:
# step size in our algorithm
delta_cs = [100, 4]
delta_npj = [200, 4]

# calculate the relative parameters change in W from the k4 to final k-point grid
# normalized to the step size of the respective convergence algorithm
w_diff_cs = []
w_diff_npj = []
for r in res_cs:
    if not -1 in r.params[:, 0]:
        temp_w_diff_cs = (r.params[3, 1:] - r.params[-1, 1:]) / delta_cs
        w_diff_cs.append(temp_w_diff_cs)
    else:
        print(f"Skipping: {r.id} ({r.name})")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_cs[0] < 0:
        print(f"CS Negative N_b:    {r.id:<10s} {r.name:<10s} {temp_w_diff_cs[0]}")
    if temp_w_diff_cs[1] < 0:
        print(f"CS Negative G_cut:  {r.id:<10s} {r.name:<10s} {temp_w_diff_cs[1]}")
for r in res_npj:
    if not -1 in r.params[:, 0]:
        temp_w_diff_npj = (r.params[3, 1:] - r.params[-1, 1:]) / delta_npj
        w_diff_npj.append(temp_w_diff_npj)
    else:
        print(f"Skipping: {r.id} ({r.name})")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_npj[0] < 0:
        print(f"NPJ Negative N_b:   {r.id:<10s} {r.name:<10s} {temp_w_diff_npj[0]}")
    if temp_w_diff_npj[1] < 0:
        print(f"NPJ Negative G_cut: {r.id:<10s} {r.name:<10s} {temp_w_diff_npj[1]}")
w_diff_cs = np.array(w_diff_cs)
w_diff_npj = np.array(w_diff_npj)

In [ ]:
# figure size (tested)
width = 2 * 2.65
height = 4.5

# text size
fs = 10

# figure
fig, axs = plt.subplots(2, 2, figsize=(width, height), facecolor="w", dpi=600)

# add (a), (b), (c) and (d) to the subplots
axs[0, 0].text(-0.2, 1.15, r"\textbf{a}", transform=axs[0, 0].transAxes, size=11)
axs[0, 1].text(-0.18, 1.15, r"\textbf{b}", transform=axs[0, 1].transAxes, size=11)
axs[1, 0].text(-0.2, 1.15, r"\textbf{c}", transform=axs[1, 0].transAxes, size=11)
axs[1, 1].text(-0.18, 1.15, r"\textbf{d}", transform=axs[1, 1].transAxes, size=11)

# plot the change in the number of bands when comparing the 4th k-grid convergence to the last k-grid
axs[0, 0].hist(
    w_diff_cs[:, 0],
    bins=np.arange(np.min(w_diff_cs[:, 0]), np.max(w_diff_cs[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 0].axvline(x=np.mean(w_diff_cs[:, 0]), color="magenta", linestyle="--")
axs[0, 0].axvline(x=np.median(w_diff_cs[:, 0]), color="limegreen", linestyle="--")
axs[0, 1].hist(
    w_diff_npj[:, 0],
    bins=np.arange(np.min(w_diff_npj[:, 0]), np.max(w_diff_npj[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 1].axvline(x=np.mean(w_diff_npj[:, 0]), color="magenta", linestyle="--")
axs[0, 1].axvline(x=np.median(w_diff_npj[:, 0]), color="limegreen", linestyle="--")

# plot the change in the cutoff when comparing the 4th k-grid convergence to the last k-grid
axs[1, 0].hist(
    w_diff_cs[:, 1],
    bins=np.arange(np.min(w_diff_cs[:, 1]), np.max(w_diff_cs[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 0].axvline(x=np.mean(w_diff_cs[:, 1]), color="magenta", linestyle="--")
axs[1, 0].axvline(x=np.median(w_diff_cs[:, 1]), color="limegreen", linestyle="--")
axs[1, 1].hist(
    w_diff_npj[:, 1],
    bins=np.arange(np.min(w_diff_npj[:, 1]), np.max(w_diff_npj[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 1].axvline(x=np.mean(w_diff_npj[:, 1]), color="magenta", linestyle="--")
axs[1, 1].axvline(x=np.median(w_diff_npj[:, 1]), color="limegreen", linestyle="--")

# titles
axs[0, 0].set_title(r"CS", fontsize=fs + 2, pad=15)
axs[0, 1].set_title(r"SOTA", fontsize=fs + 2, pad=15)

# axis labels
axs[0, 0].set_xlabel(r"$C^{k_4}_{N_b}$", size=fs)
axs[0, 1].set_xlabel(r"$C^{k_4}_{N_b}$", size=fs)
axs[0, 0].set_ylabel(r"Occurrences", size=fs)
axs[1, 0].set_xlabel(r"$C^{k_4}_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 1].set_xlabel(r"$C^{k_4}_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 0].set_ylabel(r"Occurrences", size=fs)

# axis ticks
axs[0, 0].xaxis.set_tick_params(labelsize=fs)
axs[0, 0].yaxis.set_tick_params(labelsize=fs)
axs[0, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 0].yaxis.set_ticks(axs[0, 0].get_yticks().astype(int))
axs[0, 0].xaxis.set_ticks([-3, -2, -1, 0, 1, 2, 3])
axs[0, 0].set_xlim([-3, 3])
axs[0, 1].xaxis.set_tick_params(labelsize=fs)
axs[0, 1].yaxis.set_tick_params(labelsize=fs)
axs[0, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 1].yaxis.set_ticks(axs[0, 1].get_yticks().astype(int))
axs[0, 1].xaxis.set_ticks([-3, -2, -1, 0, 1, 2, 3])
axs[0, 1].set_xlim([-3, 3])
axs[1, 0].xaxis.set_tick_params(labelsize=fs)
axs[1, 0].yaxis.set_tick_params(labelsize=fs)
axs[1, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 0].set_xlim([-3, 6])
axs[1, 0].xaxis.set_ticks([-2, 0, 2, 4, 6])
axs[1, 0].yaxis.set_ticks([0, 10, 20, 30, 40, 50, 60])
axs[1, 1].xaxis.set_tick_params(labelsize=fs)
axs[1, 1].yaxis.set_tick_params(labelsize=fs)
axs[1, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 1].set_xlim([-3, 6])
axs[1, 1].xaxis.set_ticks([-2, 0, 2, 4, 6])
axs[1, 1].yaxis.set_ticks([0, 10, 20, 30, 40, 50])

# fix layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.6, wspace=0.25)

# save the figure
fig.savefig("plots/supplement/param_indep_w_k_4.pdf")

In [ ]:
# check the percentage of how many times the k4 grid is > 0,
# i.e. that the convergence surface is more flat on the final k-point grid compared to the gamma-only grid
print("CS: C_p >= 0")
print(
    f"{len(w_diff_cs[(w_diff_cs[:,0] >= 0) & (w_diff_cs[:,1] >= 0), 0]) / len(w_diff_cs[:,0]) * 100:.2f} %\n"
)
print("NPJ: C_p >= 0")
print(
    f"{len(w_diff_npj[(w_diff_npj[:,0] >= 0) & (w_diff_npj[:,1] >= 0), 0]) / len(w_diff_npj[:,0]) * 100:.2f} %\n"
)
# we need to check where C_P >= 0 holds for both parameters at the same time

# some statistics for the parameter stability of W with respect to the k-point grid density
print("CS: MEAN")
print(f"N_b:   {np.mean(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_cs[:,1]):.1f}\n")

print("CS: MEDIAN")
print(f"N_b:   {np.median(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_cs[:,1]):.1f}\n")

print("CS: RANGE")
print(f"N_b:   {np.max(w_diff_cs[:,0]) - np.min(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_cs[:,1]) - np.min(w_diff_cs[:,1]):.1f}\n")

print("NPJ: MEAN")
print(f"N_b:   {np.mean(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_npj[:,1]):.1f}\n")

print("NPJ: MEDIAN")
print(f"N_b:   {np.median(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_npj[:,1]):.1f}\n")

print("NPJ: RANGE")
print(f"N_b:   {np.max(w_diff_npj[:,0]) - np.min(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_npj[:,1]) - np.min(w_diff_npj[:,1]):.1f}")

In [ ]:
# we investigate the average speedup by converging the
# k-grid with low W parameters compared with the W parameters from the CS convergence
# we note that this is just a approximation as the W parameters at last point of
# the CS convergence algorithm may change with the k-point grid, however by looking
# at C_p we know that this happens in about 10 % of the cases
# however, by looking at C_P for different k-point grids this effect should average out,
# since the distribution of C_P is roughly symmetric around zero.
k_speed_up = []
for r in res_cs:
    temp_t_min = 0
    temp_t_cs = 0
    for g in r.gwc:
        temp_t_min += g.grid_time[0]
        temp_t_cs += g.grid_time[-1]
    k_speed_up.append(temp_t_cs / temp_t_min)
print(
    f"Average speed up for k-grid convergence with low W parameters: {np.mean(k_speed_up):.2f}"
)
print(
    f"Median speed up for k-grid convergence with low W parameters:  {np.median(k_speed_up):.2f}"
)

# figure size (tested)
width = 2.65
height = 2.0

# text size
fs = 10

# figure
fig, ax = plt.subplots(1, 1, figsize=(width, height), facecolor="w", dpi=600)

# plot the relative time differences
ax.hist(
    k_speed_up,
    bins=np.arange(np.min(k_speed_up), np.max(k_speed_up), 1) - 0.5,
    color="tab:blue",
)
ax.axvline(x=np.mean(k_speed_up), color="magenta", linestyle="--")
ax.axvline(x=np.median(k_speed_up), color="limegreen", linestyle="--")
ax.axvline(x=1, color="k", linestyle="--")

# axis ticks
ax.xaxis.set_tick_params(labelsize=fs)
ax.yaxis.set_tick_params(labelsize=fs)
ax.tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
ax.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)
ax.xaxis.set_ticks([0, 10, 20, 30, 40])
ax.yaxis.set_ticks([0, 4, 8, 12])

# axis labels
ax.set_xlabel(r"$\textrm{SP}_\mathbf{k}$", size=fs)
ax.set_ylabel(r"Occurrences", size=fs)

# fix layout
fig.tight_layout()

# save the figure
fig.savefig("plots/supplement/k_speed_up.pdf")

In [ ]:
# we investigate the average speedup by converging the
# W parameters on a gamma-only calculation compared to the 2x2x2 k-point grid
# proposed by van Setten et al.
w_speed_up = []
for r in res_cs:
    w_speed_up.append(np.sum(r.gwc[1].grid_time) / np.sum(r.gwc[0].grid_time))
print(
    f"Average speed up for the W parameter convergence on a gamma-only grid: {np.mean(w_speed_up):.2f}"
)
print(
    f"Median speed up for the W parameter convergence on a gamma-only grid:  {np.median(w_speed_up):.2f}"
)

# figure size (tested)
width = 2.65
height = 2.0

# text size
fs = 10

# figure
fig, ax = plt.subplots(1, 1, figsize=(width, height), facecolor="w", dpi=600)

# plot the relative time differences
ax.hist(
    w_speed_up,
    bins=np.arange(np.min(w_speed_up), np.max(w_speed_up), 0.1) - 0.5,
    color="tab:blue",
)
ax.axvline(x=np.mean(w_speed_up), color="magenta", linestyle="--")
ax.axvline(x=np.median(w_speed_up), color="limegreen", linestyle="--")
ax.axvline(x=1, color="k", linestyle="--")

# axis ticks
ax.xaxis.set_tick_params(labelsize=fs)
ax.yaxis.set_tick_params(labelsize=fs)
ax.tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
ax.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)
ax.xaxis.set_ticks([0, 2, 4, 6, 8])
ax.yaxis.set_ticks([0, 2, 4, 6, 8])

# axis limit
ax.set_xlim(0, 8)

# axis labels
ax.set_xlabel(
    r"$\textrm{T}^{2\times 2\times 2}_\textrm{\scriptsize CS}/\textrm{T}^\Gamma_\textrm{\scriptsize CS}$",
    size=fs,
)
ax.set_ylabel(r"Occurrences", size=fs)

# fix layout
fig.tight_layout()

# save the figure
fig.savefig("plots/supplement/w_speed_up.pdf")

In [ ]:
# setup publication plot size
width = 6
height = 8

# text size settings
tick_size = int(width * 1.75)
label_size = int(width * 1.75)

In [ ]:
# create a plot for the difference between the optimal gap found by the convergence
# algorithms and the reference calculations, which were performed with very high computation parameters
# material (1-20)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20):
    idx_used_cs = res_cs[i].gap > 0
    idx_used_npj = res_npj[i].gap > 0
    axs[i].plot(
        np.where(idx_used_cs)[0],
        1000 * np.abs(res_cs[i].gap[idx_used_cs] - res_ref[i].gap[idx_used_cs]),
        "ks--",
        markersize=4,
    )
    axs[i].plot(
        np.where(idx_used_npj)[0],
        1000 * np.abs(res_npj[i].gap[idx_used_npj] - res_ref[i].gap[idx_used_npj]),
        "ro--",
        markersize=4,
    )
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[i].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
    axs[i].axhline(y=conv_thr, color="b", linestyle="-", zorder=-1)
    ylimit = axs[i].get_ylim()
    axs[i].set_ylim([0, ylimit[1] * 1.05])
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(
    r"$\vert\Delta_\textrm{R}\textrm{E}_\textrm{gap}^{{\Gamma}-{\Gamma}}\vert$ (meV)",
    x=0.025,
    y=0.55,
    size=label_size,
)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_gap_diff_ref_1.pdf")

# create a plot for the difference between the optimal gap found by the convergence
# algorithms and the reference calculations, which were performed with very high computation parameters
# material (21-40)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20, 40):
    axis_idx = i - 20
    idx_used_cs = res_cs[i].gap > 0
    idx_used_npj = res_npj[i].gap > 0
    axs[axis_idx].plot(
        np.where(idx_used_cs)[0],
        1000 * np.abs(res_cs[i].gap[idx_used_cs] - res_ref[i].gap[idx_used_cs]),
        "ks--",
        markersize=4,
    )
    axs[axis_idx].plot(
        np.where(idx_used_npj)[0],
        1000 * np.abs(res_npj[i].gap[idx_used_npj] - res_ref[i].gap[idx_used_npj]),
        "ro--",
        markersize=4,
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[axis_idx].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].axhline(y=conv_thr, color="b", linestyle="-", zorder=-1)
    ylimit = axs[axis_idx].get_ylim()
    axs[axis_idx].set_ylim([0, ylimit[1] * 1.05])
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(
    r"$\vert\Delta_\textrm{R}\textrm{E}_\textrm{gap}^{{\Gamma}-{\Gamma}}\vert$ (meV)",
    x=0.025,
    y=0.55,
    size=label_size,
)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_gap_diff_ref_2.pdf")

# create a plot for the difference between the optimal gap found by the convergence
# algorithms and the reference calculations, which were performed with very high computation parameters
# material (41-60)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(40, 60):
    axis_idx = i - 40
    idx_used_cs = res_cs[i].gap > 0
    idx_used_npj = res_npj[i].gap > 0
    axs[axis_idx].plot(
        np.where(idx_used_cs)[0],
        1000 * np.abs(res_cs[i].gap[idx_used_cs] - res_ref[i].gap[idx_used_cs]),
        "ks--",
        markersize=4,
    )
    axs[axis_idx].plot(
        np.where(idx_used_npj)[0],
        1000 * np.abs(res_npj[i].gap[idx_used_npj] - res_ref[i].gap[idx_used_npj]),
        "ro--",
        markersize=4,
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[axis_idx].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].axhline(y=conv_thr, color="b", linestyle="-", zorder=-1)
    ylimit = axs[axis_idx].get_ylim()
    axs[axis_idx].set_ylim([0, ylimit[1] * 1.05])
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(
    r"$\vert\Delta_\textrm{R}\textrm{E}_\textrm{gap}^{{\Gamma}-{\Gamma}}\vert$ (meV)",
    x=0.025,
    y=0.55,
    size=label_size,
)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_gap_diff_ref_3.pdf")

In [ ]:
# create a plot for the number of gw calculation per convergence calculation
# (material 1-20)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20):
    idx_used_cs = res_cs[i].ngw > 0
    idx_used_npj = res_npj[i].ngw > 0
    axs[i].plot(
        np.where(idx_used_cs)[0], res_cs[i].ngw[idx_used_cs], "ks--", markersize=4
    )
    axs[i].plot(
        np.where(idx_used_npj)[0], res_npj[i].ngw[idx_used_npj], "ro--", markersize=4
    )
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[i].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_ticks(axs[i].get_yticks().astype(int))
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{N}_\textrm{GW}$", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_ngw_1.pdf")

# create a plot for the number of gw calculation per convergence calculation
# (material 21-40)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20, 40):
    axis_idx = i - 20
    idx_used_cs = res_cs[i].ngw > 0
    idx_used_npj = res_npj[i].ngw > 0
    axs[axis_idx].plot(
        np.where(idx_used_cs)[0], res_cs[i].ngw[idx_used_cs], "ks--", markersize=4
    )
    axs[axis_idx].plot(
        np.where(idx_used_npj)[0], res_npj[i].ngw[idx_used_npj], "ro--", markersize=4
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[axis_idx].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_ticks(axs[axis_idx].get_yticks().astype(int))
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{N}_\textrm{GW}$", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_ngw_2.pdf")

# create a plot for the number of gw calculation per convergence calculation
# (material 41-60)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(40, 60):
    axis_idx = i - 40
    idx_used_cs = res_cs[i].ngw > 0
    idx_used_npj = res_npj[i].ngw > 0
    axs[axis_idx].plot(
        np.where(idx_used_cs)[0], res_cs[i].ngw[idx_used_cs], "ks--", markersize=4
    )
    axs[axis_idx].plot(
        np.where(idx_used_npj)[0], res_npj[i].ngw[idx_used_npj], "ro--", markersize=4
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[axis_idx].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_ticks(axs[axis_idx].get_yticks().astype(int))
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{N}_\textrm{GW}$", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_ngw_3.pdf")

In [ ]:
# create a plot for the computation time for each convergence calculations
# (material 1-20)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20):
    idx_used_cs = res_cs[i].tcpu > 1
    idx_used_npj = res_npj[i].tcpu > 1
    axs[i].plot(
        np.where(idx_used_cs)[0],
        res_cs[i].tcpu[idx_used_cs] / 3600,
        "ks--",
        markersize=4,
    )
    axs[i].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].tcpu[idx_used_npj] / 3600,
        "ro--",
        markersize=4,
    )
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[i].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
    axs[i].set_yscale("log")
    y_minor = mticker.LogLocator(
        base=10.0, subs=np.arange(1.0, 10.0) * 0.1, numticks=10
    )
    y_major = mticker.LogLocator(base=10.0, numticks=5)
    axs[i].yaxis.set_minor_locator(y_minor)
    axs[i].yaxis.set_minor_formatter(mticker.NullFormatter())
    axs[i].yaxis.set_major_locator(y_major)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{T}_\textrm{CPU}$ (h)", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_tcpu_1.pdf")

# create a plot for the computation time for each convergence calculations
# (material 21-40)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20, 40):
    axis_idx = i - 20
    idx_used_cs = res_cs[i].tcpu > 1
    idx_used_npj = res_npj[i].tcpu > 1
    axs[axis_idx].plot(
        np.where(idx_used_cs)[0],
        res_cs[i].tcpu[idx_used_cs] / 3600,
        "ks--",
        markersize=4,
    )
    axs[axis_idx].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].tcpu[idx_used_npj] / 3600,
        "ro--",
        markersize=4,
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[axis_idx].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].set_yscale("log")
    y_minor = mticker.LogLocator(
        base=10.0, subs=np.arange(1.0, 10.0) * 0.1, numticks=10
    )
    y_major = mticker.LogLocator(base=10.0, numticks=5)
    axs[axis_idx].yaxis.set_minor_locator(y_minor)
    axs[axis_idx].yaxis.set_minor_formatter(mticker.NullFormatter())
    axs[axis_idx].yaxis.set_major_locator(y_major)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{T}_\textrm{CPU}$ (h)", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_tcpu_2.pdf")

# create a plot for the computation time for each convergence calculations
# (material 41-60)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(40, 60):
    axis_idx = i - 40
    idx_used_cs = res_cs[i].tcpu > 1
    idx_used_npj = res_npj[i].tcpu > 1
    axs[axis_idx].plot(
        np.where(idx_used_cs)[0],
        res_cs[i].tcpu[idx_used_cs] / 3600,
        "ks--",
        markersize=4,
    )
    axs[axis_idx].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].tcpu[idx_used_npj] / 3600,
        "ro--",
        markersize=4,
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[axis_idx].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].set_yscale("log")
    y_minor = mticker.LogLocator(
        base=10.0, subs=np.arange(1.0, 10.0) * 0.1, numticks=10
    )
    y_major = mticker.LogLocator(base=10.0, numticks=5)
    axs[axis_idx].yaxis.set_minor_locator(y_minor)
    axs[axis_idx].yaxis.set_minor_formatter(mticker.NullFormatter())
    axs[axis_idx].yaxis.set_major_locator(y_major)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{T}_\textrm{CPU}$ (h)", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_tcpu_3.pdf")

In [ ]:
# create a plot for the convergence parameter N_b for each convergence calculation
# (material 1-20)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20):
    idx_used_cs = res_cs[i].params[:, 0] > 0
    idx_used_npj = res_npj[i].params[:, 0] > 0
    axs[i].plot(
        np.where(idx_used_cs)[0], res_cs[i].params[idx_used_cs, 0], "ks--", markersize=4
    )
    axs[i].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].params[idx_used_npj, 0],
        "ro--",
        markersize=4,
    )
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[i].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[i].yaxis.set_ticks(
        np.sort(
            np.unique(
                np.append(
                    res_cs[i].params[idx_used_cs, 0], res_npj[i].params[idx_used_npj, 0]
                )
            )
        )
    )
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{N}_\textrm{b}$", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_nbx_1.pdf")

# create a plot for the convergence parameter N_b for each convergence calculation
# (material 21-40)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20, 40):
    axis_idx = i - 20
    idx_used_cs = res_cs[i].params[:, 0] > 0
    idx_used_npj = res_npj[i].params[:, 0] > 0
    axs[axis_idx].plot(
        np.where(idx_used_cs)[0], res_cs[i].params[idx_used_cs, 0], "ks--", markersize=4
    )
    axs[axis_idx].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].params[idx_used_npj, 0],
        "ro--",
        markersize=4,
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[axis_idx].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[axis_idx].yaxis.set_ticks(
        np.sort(
            np.unique(
                np.append(
                    res_cs[i].params[idx_used_cs, 0], res_npj[i].params[idx_used_npj, 0]
                )
            )
        )
    )
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{N}_\textrm{b}$", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_nbx_2.pdf")

# create a plot for the convergence parameter N_b for each convergence calculation
# (material 41-60)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(40, 60):
    axis_idx = i - 40
    idx_used_cs = res_cs[i].params[:, 0] > 0
    idx_used_npj = res_npj[i].params[:, 0] > 0
    axs[axis_idx].plot(
        np.where(idx_used_cs)[0], res_cs[i].params[idx_used_cs, 0], "ks--", markersize=4
    )
    axs[axis_idx].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].params[idx_used_npj, 0],
        "ro--",
        markersize=4,
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[axis_idx].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[axis_idx].yaxis.set_ticks(
        np.sort(
            np.unique(
                np.append(
                    res_cs[i].params[idx_used_cs, 0], res_npj[i].params[idx_used_npj, 0]
                )
            )
        )
    )
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{N}_\textrm{b}$", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_nbx_3.pdf")

In [ ]:
# create a plot for the convergence parameter G_cut for each convergence calculation
# (material 1-20)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20):
    idx_used_cs = res_cs[i].params[:, 2] > 0
    idx_used_npj = res_npj[i].params[:, 2] > 0
    axs[i].plot(
        np.where(idx_used_cs)[0], res_cs[i].params[idx_used_cs, 2], "ks--", markersize=4
    )
    axs[i].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].params[idx_used_npj, 2],
        "ro--",
        markersize=4,
    )
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[i].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[i].yaxis.set_ticks(
        np.sort(
            np.unique(
                np.append(
                    res_cs[i].params[idx_used_cs, 2], res_npj[i].params[idx_used_npj, 2]
                )
            )
        )
    )
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$G_\textrm{cut}$ (Ry)", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_sc_1.pdf")

# create a plot for the convergence parameter G_cut for each convergence calculation
# (material 21-40)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20, 40):
    axis_idx = i - 20
    idx_used_cs = res_cs[i].params[:, 2] > 0
    idx_used_npj = res_npj[i].params[:, 2] > 0
    axs[axis_idx].plot(
        np.where(idx_used_cs)[0], res_cs[i].params[idx_used_cs, 2], "ks--", markersize=4
    )
    axs[axis_idx].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].params[idx_used_npj, 2],
        "ro--",
        markersize=4,
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[axis_idx].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[axis_idx].yaxis.set_ticks(
        np.sort(
            np.unique(
                np.append(
                    res_cs[i].params[idx_used_cs, 2], res_npj[i].params[idx_used_npj, 2]
                )
            )
        )
    )
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$G_\textrm{cut}$ (Ry)", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_sc_2.pdf")

# create a plot for the convergence parameter G_cut for each convergence calculation
# (material 41-60)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(40, 60):
    axis_idx = i - 40
    idx_used_cs = res_cs[i].params[:, 2] > 0
    idx_used_npj = res_npj[i].params[:, 2] > 0
    axs[axis_idx].plot(
        np.where(idx_used_cs)[0], res_cs[i].params[idx_used_cs, 2], "ks--", markersize=4
    )
    axs[axis_idx].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].params[idx_used_npj, 2],
        "ro--",
        markersize=4,
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[axis_idx].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[axis_idx].yaxis.set_ticks(
        np.sort(
            np.unique(
                np.append(
                    res_cs[i].params[idx_used_cs, 2], res_npj[i].params[idx_used_npj, 2]
                )
            )
        )
    )
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$G_\textrm{cut}$ (Ry)", x=0.025, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_sc_3.pdf")

In [ ]:
# plot the gap difference for the k-point convergence to show the independence
# of the k-point mesh with respect to the parameters in W
# material (1-20)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20):
    temp_gaps = np.zeros(5)
    for j in range(5):
        temp_gaps[j] = res_cs[i].gwc[j].grid[0, 2]
    axs[i].plot(list(range(4)), np.abs(np.diff(temp_gaps)), "ks--", markersize=4)
    axs[i].plot(list(range(4)), np.abs(np.diff(res_ref[i].gap)), "ro--", markersize=4)
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].set_xticks([0, 1, 2, 3])
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa) - 1)))
    axs[i].xaxis.set_ticklabels([2, 3, 4, 5])
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
    axs[i].set_yscale("log")
    axs[i].axhline(y=0.025, color="b", linestyle="-", zorder=-1)
    y_minor = mticker.LogLocator(
        base=10.0, subs=np.arange(1.0, 10.0) * 0.1, numticks=10
    )
    y_major = mticker.LogLocator(base=10.0, numticks=5)
    axs[i].yaxis.set_minor_locator(y_minor)
    axs[i].yaxis.set_minor_formatter(mticker.NullFormatter())
    axs[i].yaxis.set_major_locator(y_major)
fig.supxlabel(r"k-point grid index $i$", x=0.55, y=0.020, size=label_size)
fig.supylabel(
    r"$\vert\Delta_{\mathbf{k}_i}\textrm{E}_\textrm{gap}^{{\Gamma}-{\Gamma}}\vert$ (eV)",
    x=0.025,
    y=0.55,
    size=label_size,
)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_gap_diff_kpt_1.pdf")

# plot the gap difference for the k-point convergence to show the independence
# of the k-point mesh with respect to the parameters in W
# material (21-40)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(20, 40):
    axis_idx = i - 20
    temp_gaps = np.zeros(5)
    for j in range(5):
        temp_gaps[j] = res_cs[i].gwc[j].grid[0, 2]
    axs[axis_idx].plot(list(range(4)), np.abs(np.diff(temp_gaps)), "ks--", markersize=4)
    axs[axis_idx].plot(
        list(range(4)), np.abs(np.diff(res_ref[i].gap)), "ro--", markersize=4
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].set_xticks([0, 1, 2, 3])
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa) - 1)))
    axs[axis_idx].xaxis.set_ticklabels([2, 3, 4, 5])
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].set_yscale("log")
    axs[axis_idx].axhline(y=0.025, color="b", linestyle="-", zorder=-1)
    y_minor = mticker.LogLocator(
        base=10.0, subs=np.arange(1.0, 10.0) * 0.1, numticks=10
    )
    y_major = mticker.LogLocator(base=10.0, numticks=5)
    axs[axis_idx].yaxis.set_minor_locator(y_minor)
    axs[axis_idx].yaxis.set_minor_formatter(mticker.NullFormatter())
    axs[axis_idx].yaxis.set_major_locator(y_major)
fig.supxlabel(r"k-point grid index $i$", x=0.55, y=0.020, size=label_size)
fig.supylabel(
    r"$\vert\Delta_{\mathbf{k}_i}\textrm{E}_\textrm{gap}^{{\Gamma}-{\Gamma}}\vert$ (eV)",
    x=0.025,
    y=0.55,
    size=label_size,
)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_gap_diff_kpt_2.pdf")

# plot the gap difference for the k-point convergence to show the independence
# of the k-point mesh with respect to the parameters in W
# material (41-60)
fig, axs = plt.subplots(5, 4, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(40, 60):
    axis_idx = i - 40
    temp_gaps = np.zeros(5)
    for j in range(5):
        temp_gaps[j] = res_cs[i].gwc[j].grid[0, 2]
    axs[axis_idx].plot(list(range(4)), np.abs(np.diff(temp_gaps)), "ks--", markersize=4)
    axs[axis_idx].plot(
        list(range(4)), np.abs(np.diff(res_ref[i].gap)), "ro--", markersize=4
    )
    axs[axis_idx].set_title(f"{res_cs[i].name}")
    axs[axis_idx].set_xticks([0, 1, 2, 3])
    axs[axis_idx].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa) - 1)))
    axs[axis_idx].xaxis.set_ticklabels([2, 3, 4, 5])
    axs[axis_idx].xaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].yaxis.set_tick_params(labelsize=tick_size)
    axs[axis_idx].set_yscale("log")
    axs[axis_idx].axhline(y=0.025, color="b", linestyle="-", zorder=-1)
    y_minor = mticker.LogLocator(
        base=10.0, subs=np.arange(1.0, 10.0) * 0.1, numticks=10
    )
    y_major = mticker.LogLocator(base=10.0, numticks=5)
    axs[axis_idx].yaxis.set_minor_locator(y_minor)
    axs[axis_idx].yaxis.set_minor_formatter(mticker.NullFormatter())
    axs[axis_idx].yaxis.set_major_locator(y_major)
fig.supxlabel(r"k-point grid index $i$", x=0.55, y=0.020, size=label_size)
fig.supylabel(
    r"$\vert\Delta_{\mathbf{k}_i}\textrm{E}_\textrm{gap}^{{\Gamma}-{\Gamma}}\vert$ (eV)",
    x=0.025,
    y=0.55,
    size=label_size,
)
fig.tight_layout()
fig.savefig("plots/supplement/gw_conv_gap_diff_kpt_3.pdf")